# Challenge 03: Data Modelling: From Retrieval to Upload (1/2)


In this step, we will structure the data retrieved from Azure Document Intelligence (ADI) into the right format to be read by our systems in subsequent steps. 

The data will be outputted from the ADI as a JSON file, and it is our role to process and organize it. Some of the data will be structured into tables, while other data will be formatted as text. This step ensures that the extracted information is organized in a meaningful way for further analysis and usage.

As stated before, we need to make sure that our Function will know how to process:
- **Loan Forms:** Extract relevant details such as borrower information, loan amounts, and terms.
- **Loan Contract:** Identify and parse key contract elements like clauses, signatures, and dates.
- **Pay Stubs:** Retrieve data such as employee details, earnings, deductions, and net pay.

Not all customers will have provided all types of content, and during this Challenge we will be only be processing one file. We will combine in the next challenge the capabilities of a trigger, which will, at a time, also process one single document.

Due to the nature of this challenge, we will separate this challenge in the 3 different types of documents.

## Loan Forms 

The first step to get a Loan, is to fill out a form with some basic details, such as customer ID, Full Name, Date Of Birth, etc, therefore, that's where we will start. 

This particular document combines text and tables, that the ADI capabilities allow you to extract as also separate capabilities.

To first start our analysis, let's create a function that will load the documents inside a folder inside a container that is, on its turn, inside our designated Storage Account. In our particular step, inside the folder of the Loan Forms, we will retrieve one Loan Form for us to analyse. 

We will consequently use this same function to access other folders that will contain other type of documents.

**Question: why are we not batch-analysing documents?**





In [1]:
import os
import json
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def read_json_files_from_blob(folder_path):
    # Retrieve the connection string from the environment variables
    connection_string = os.getenv('connection_string')

    # Ensure the connection string is not None
    if connection_string is None:
        raise ValueError("The connection string environment variable is not set.")

    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get the container client
    container_client = blob_service_client.get_container_client("bankdetail")

    # List all blobs in the specified folder
    blob_list = container_client.list_blobs(name_starts_with=folder_path)

    # Filter out JSON files and read their contents
    for blob in blob_list:
        if blob.name.endswith('.json'):
            blob_client = container_client.get_blob_client(blob.name)
            blob_data = blob_client.download_blob().readall()
            data = json.loads(blob_data)
            # print(f"Contents of {blob.name}:")
            # print(json.dumps(data, indent=2))
            # print("\n")
            return data 

Now all we have to do is to call our function and pass the name of our folder as an argument

In [2]:
loanform = read_json_files_from_blob("loanform") ## RETIRAR PARA ELES PERCEBEREM OQ TAO A FAZER

The next step is to create a function that will process the loan application form data. This function will take the loan application form data as input and return the result of the loan application processing. Our input data is a JSON file that is composed of both text and tables, and we will need to treat both of them seperatly.  

The function will perform the following steps:

The create_structured_tables function processes a list of tables by initializing and populating them with cell content, combining specific rows for tables with 3 rows and 5 columns, and returning the structured tables along with any combined rows.

In [3]:
def create_structured_tables(tables):
    structured_tables = []
    combined_rows = []
    
    for table in tables:
        row_count = table.get("row_count", 0)
        column_count = table.get("column_count", 0)
        cells = table.get("cells", [])
        
        # Initialize an empty table
        structured_table = [["" for _ in range(column_count)] for _ in range(row_count)]
        
        # Populate the table with cell content
        for cell in cells:
            row_index = cell.get("row_index", 0)
            column_index = cell.get("column_index", 0)
            content = cell.get("content", "")
            structured_table[row_index][column_index] = content
        
        # Combine the last row with the previous one if the table has 5 columns and 3 rows
        if row_count == 3 and column_count == 5:
            combined_row = [structured_table[1][i] + " " + structured_table[2][i] for i in range(column_count)]
            structured_table[1] = combined_row
            structured_table = structured_table[:2]
            combined_rows.append(combined_row)
        
        # Append the structured table to the list
        structured_tables.append(structured_table)
    
    return structured_tables, combined_rows

The clean_form_recognizer_result function processes form recognizer output by extracting text data while ignoring lines containing the word "table", retaining only the "text" key in each line, and creating structured tables from the table data.

In [4]:
def clean_form_recognizer_result(data):
    text_data = []
    table_encountered = False
    
    for page in data.get("pages", []):
        for line in page.get("lines", []):
            # Check if the line contains the word "table"
            if "table" in line.get("text", "").lower():
                table_encountered = True
                continue  # Skip the line if "table" is in the text
            
            if not table_encountered:
                # Collect the "text" information
                text_data.append(line.get("text", ""))
            
            # Keep only the "text" key
            line_keys = list(line.keys())
            for key in line_keys:
                if key != "text":
                    del line[key]
    
    # Create structured tables
    structured_tables, combined_rows = create_structured_tables(data.get("tables", []))
    data["structured_tables"] = structured_tables
    data["combined_rows"] = combined_rows
    data["text_data"] = text_data
    
    return data

The tables_to_dataframes function converts a list of structured tables into a list of pandas DataFrames.

In [5]:
def tables_to_dataframes(structured_tables):
    dataframes = []
    for table in structured_tables:
        df = pd.DataFrame(table)
        dataframes.append(df)
    return dataframes

We have now retrieved both our table with the structured desired and the text that comes out of our files. However, this function doesn't have the data as structured as we need it to be. 

As an example, we have by now extracted a key-value pair which keys is "text" with the value "Contact Number: (555) 234-5678". What we will need to define now is to remove the name of the field, and start composing the key-value pair that would be key "Contact Number:" and value "(555) 234-5678"

In [7]:
import pandas as pd
import re 

def clean_loan_application_file(text):
    cleaned_data = {}

    # Extract the category from the first three words
    category_match = re.search(r'(\w+\s+\w+\s+\w+)', text)
    if category_match:
        cleaned_data['Category'] = category_match.group(1)
    
    # Extract Applicant Information
    applicant_info = re.search(r'Applicant Information(.*?)Employment and Income Details', text, re.DOTALL)
    if applicant_info:
        applicant_info_text = applicant_info.group(1)
        cleaned_data['Applicant Information'] = {
            'Full Name': re.search(r'Full Name:\s*(.*?)Date of Birth:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Date of Birth': re.search(r'Date of Birth:\s*(.*?)Social Security Number:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Social Security Number': re.search(r'Social Security Number:\s*(.*?)Contact Number:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Contact Number': re.search(r'Contact Number:\s*(.*?)Email Address:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Email Address': re.search(r'Email Address:\s*(.*?)Physical Address:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Physical Address': re.search(r'Physical Address:\s*(.*)', applicant_info_text, re.DOTALL).group(1).strip(),
        }

    # Extract Loan Information
    loan_info = re.search(r'Loan Information(.*)', text, re.DOTALL)
    if loan_info:
        loan_info_text = loan_info.group(1)
        cleaned_data['Loan Information'] = {
            'Loan Amount Requested': re.search(r'Loan Amount Requested:\s*\$?(.*?)Purpose of Loan:', loan_info_text, re.DOTALL).group(1).strip(),
            'Purpose of Loan': re.search(r'Purpose of Loan:\s*(.*?)Loan Term Desired:', loan_info_text, re.DOTALL).group(1).strip(),
            'Loan Term Desired': re.search(r'Loan Term Desired:\s*(.*)', loan_info_text, re.DOTALL).group(1).strip(),
        }

    return cleaned_data

# Function to combine extracted tables and text
def process_loan_application(data):
    # Clean form recognizer result to extract structured tables and text
    cleaned_data = clean_form_recognizer_result(data)
    
    # Convert extracted tables to dataframes
    dataframes = tables_to_dataframes(cleaned_data["structured_tables"]) #tabela bem retirada
    # print(dataframes)
    # Combine all table dataframes into one
    combined_df = pd.concat(dataframes, ignore_index=True) #tabela bem retirada
    combined_df.columns = combined_df.iloc[0]
    combined_df = combined_df[1:]
    combined_df.reset_index(drop=True, inplace=True)
    combined_df.rename(columns={"Contact Number": "Employer Contact Number"}, inplace=True)
    combined_df = combined_df.dropna(how='all')

    # Clean the extracted text using regex
    combined_text = ' '.join(cleaned_data['text_data'])
    text_data = clean_loan_application_file(combined_text)
    #print(text_data) #tudo menos a tabela perfeitamente mas em dict

    def clean_loan_application(data):
    # Extract applicant and loan info
        applicant_info = data['Applicant Information']
        loan_info = data['Loan Information']
        
        # Combine keys and values for the two categories
        fields = list(applicant_info.keys()) + list(loan_info.keys())
        values = list(applicant_info.values()) + list(loan_info.values())
        
        # Create the 2x10 DataFrame without 'Category'
        df = pd.DataFrame({
            'Field': fields,
            'Value': values
        })
        
        return df.set_index('Field').T

    df_cleaned = clean_loan_application(text_data)

    # Convert the text data to a DataFrame
    text_df = pd.DataFrame(df_cleaned)

    # Concatenate the text dataframe with the tables dataframe
    final_df = pd.concat([text_df, combined_df], axis=1)

    def remove_empty_cells_and_push_up(df):
        for column in df.columns:
            non_empty_values = df[column].replace('', pd.NA).dropna().values
            df[column] = pd.Series(non_empty_values).reindex(df.index, fill_value='')
        return df

    return remove_empty_cells_and_push_up(final_df)

# Process the loan application
loanform = process_loan_application(loanform).iloc[1:].reset_index(drop=True)

# Save the final dataframe to an Excel file
# loanform.to_excel('loanform.xlsx', index=False)

## Pay Stub 

As part of some loan applications, the pay stub is a required document. The pay stub is a document that outlines the details of an employee’s income. It contains the employee’s wages earned, applicable deductions and total gross pay, and net pay for the pay period. A pay stub will provide Contoso bank with crucial information about not only a person’s income and employment stability, which helps assess their ability to repay the loan. It also verifies the applicant’s financial credibility and ensures that their reported income matches their actual earnings.

When processing a Pay Stub, we will have similar challenges as we previously did on the Loan Forms. These particular documents combine text and contrary to the previous use case, more than 1 table, Once again, the ADI capabilities allows you to extract these 2 types of entities as also separate capabilities.

As we've previously create a the function that will load the documents inside a designated folder, all we have to do now is to retrieve all the information inside the paystub folder, we will retrieve one single Loan Form for us to analyse.


In [8]:
paystub = read_json_files_from_blob("paystubs") ## RETIRAR PARA ELES PERCEBEREM OQ TAO A FAZER

In [9]:
def clean_form_recognizer_result(data):
    for page in data.get("pages", []):
        for line in page.get("lines", []):
            # Check if the line contains the word "table"
            if "table" in line.get("text", "").lower():
                continue  # Keep everything if "table" is in the text
            # Keep only the "text" key
            line_keys = list(line.keys())
            for key in line_keys:
                if key != "text":
                    del line[key]
    
    # Create structured tables
    structured_tables = create_structured_tables(data.get("tables", []))
    data["structured_tables"] = structured_tables
    
    return data

def create_structured_tables(tables):
    structured_tables = []
    for table in tables:
        row_count = table.get("row_count", 0)
        column_count = table.get("column_count", 0)
        cells = table.get("cells", [])
        
        # Initialize an empty table
        structured_table = [["" for _ in range(column_count)] for _ in range(row_count)]
        
        # Populate the table with cell content
        for cell in cells:
            row_index = cell.get("row_index", 0)
            column_index = cell.get("column_index", 0)
            content = cell.get("content", "")
            structured_table[row_index][column_index] = content
        
        structured_tables.append(structured_table)
    
    return structured_tables

import pandas as pd
def tables_to_dataframes(structured_tables):
    dataframes = []
    for table in structured_tables:
        df = pd.DataFrame(table)
        dataframes.append(df)
    return dataframes

cleaned_data = clean_form_recognizer_result(paystub)
dataframes = tables_to_dataframes(cleaned_data["structured_tables"])

df_list = []

for df in dataframes:
    print(df)
    df_list.append(df)

paystub = pd.concat(df_list, ignore_index=True)
# combined_df.to_excel('dataframes_combined.xlsx', index=False)

                0             1       2                 3  \
0     Description  Hours Worked    Rate  Current Earnings   
1     Regular Pay           160  $28.65         $4,583.33   
2    Overtime Pay             5  $42.98           $214.90   
3           Bonus           N/A     N/A           $250.00   
4  Total Earnings                               $5,048.23   

                       4  
0  Year-to-Date Earnings  
1             $32,083.31  
2              $1,289.40  
3              $1,750.00  
4             $35,122.71  
                     0               1                    2
0          Description  Current Amount  Year-to-Date Amount
1          Federal Tax         $800.00            $5,600.00
2            State Tax         $200.00            $1,400.00
3      Social Security         $314.99            $2,204.93
4             Medicare          $73.66              $515.49
5  401(k) Contribution         $250.00            $1,750.00
6     Health Insurance         $150.00            $

## Loan Agreement

Now we get to the last part of our logical set of documents on a loan application process: the final loan agreement contract has been created and signed. A loan agreement contract is a legally binding document between a lender and a borrower that outlines the terms and conditions of a loan. This contract specifies the loan amount, interest rate, repayment schedule, and any other obligations or rights of both parties. It is crucial as it provides clarity and protection for both the lender and the borrower, ensuring that both parties understand their responsibilities and the consequences of default. Additionally, it serves as a legal record that can be referenced in case of disputes, helping to prevent misunderstandings and enforce the agreed-upon terms.

The format of a loan agreement is, on its core, a text document that will not have a fixed structure. We should expect just as an input text document and therefore retrieve it as such. 

As we did in the previous steps, let's call the function that will retrieve the information inside the loanagreements folder, retrieving, once again, one single Loan Agreement.





#### Loan Agreements

In [21]:
loanagreement = read_json_files_from_blob("loanagreements")

In [22]:
def clean_json_data(json_data):
    # Extract relevant text content from the JSON
    content = []

    # Extract text from paragraphs
    paragraphs = json_data.get("paragraphs", [])
    for paragraph in paragraphs:
        content.append(paragraph.get("text", "").strip())

    # Extract text from pages and lines
    pages = json_data.get("pages", [])
    for page in pages:
        for line in page.get("lines", []):
            content.append(line.get("text", "").strip())

    # Join all text content into a single string with spaces between components
    plain_text_content = " ".join(content)

    return plain_text_content

# Clean the JSON data
loanagreement = clean_json_data(loanagreement)

# Print the cleaned data
# print(json.dumps(cleaned_data, indent=2))

# Challenge 03: Data Architecturing: From Retrieval to Upload (2/2)

Adicionar o Customer
Upload to Cosmos

In [29]:
import pandas as pd
import json
from azure.cosmos import CosmosClient, exceptions, PartitionKey
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

endpoint = os.getenv("COSMOS_DB_ENDPOINT")
key = os.getenv("COSMOS_DB_KEY")
database_name = os.getenv("COSMOS_DB_DATABASE_NAME")
container_name = os.getenv("COSMOS_DB_CONTAINER_NAME")
file_name = os.getenv("FILE_NAME")

def upload_dataframe_to_cosmos_db(df, file_name, endpoint, key, database_name, container_name):
    # Initialize the Cosmos client
    client = CosmosClient(endpoint, key)
    
    # Create or get the database
    database = client.create_database_if_not_exists(id=database_name)
    
    # Create or get the container
    container = database.create_container_if_not_exists(
        id=container_name,
        partition_key=PartitionKey(path="/id"),
        offer_throughput=400
    )
    
    # Convert DataFrame to JSON string
    json_content = df.to_json(orient='records')
    
    # Parse the JSON string
    json_data = json.loads(json_content)
    
    # Create a document with the JSON content
    document = {
        'id': file_name,
        'content': json_data
    }
    
    # Upload the document to the container
    try:
        container.create_item(body=document)
        print(f"JSON content uploaded successfully as '{file_name}' in Cosmos DB.")
    except exceptions.CosmosHttpResponseError as e:
        print(f"An error occurred: {e.message}")

# Example usage
# df = pd.read_csv('path_to_your_csv_file.csv')
upload_dataframe_to_cosmos_db(loanform, "loanform1", endpoint, key, database_name, container_name)

JSON content uploaded successfully as 'loanform1' in Cosmos DB.
